In [29]:
"""
전체적인 구조

#첫 번째 레이어
합성곱(in_channel = 1, out_channel = 32, kennel_size = 3, stride = 1, padding = 1) + 활성화 함수(Activation function)PeLU
맥스플링(kernel_size=2, stride=2)

#두 번째 레이어
합성곱(in_channel = 23, out_channel = 64, kennel_size = 3, stride = 1, padding = 1) + 활성화 함수(Activation function)PeLU

#세 번째 레이어
batch_size x 7 x 7 x 64 => batch size = 3136(특성맵을 펼치는 레이어)
Fully connected(전결합층 뉴런 10개) + 활성화 함수 softmax
"""

'\n전체적인 구조\n\n#첫 번째 레이어\n합성곱(in_channel = 1, out_channel = 32, kennel_size = 3, stride = 1, padding = 1) + 활성화 함수(Activation function)PeLU\n맥스플링(kernel_size=2, stride=2)\n\n#두 번째 레이어\n합성곱(in_channel = 23, out_channel = 64, kennel_size = 3, stride = 1, padding = 1) + 활성화 함수(Activation function)PeLU\n\n#세 번째 레이어\nbatch_size x 7 x 7 x 64 => batch size = 3136(특성맵을 펼치는 레이어)\nFully connected(전결합층 뉴런 10개) + 활성화 함수 softmax\n'

In [30]:
#구현에 필요한 라이브러리 불러오기

import torch
import torch.nn as nn
#as를 선언하지 않았을 경우 torch.nn.Conv2d() 
#as를 선언했을 경우 nn.Conv2d()

In [31]:
#임의의 텐서 생성 (1,1,28,28) => MNIST데이터
inputs = torch.Tensor(1,1,28,28)
print("텐서의 크기:()", format(inputs.shape))

텐서의 크기:() torch.Size([1, 1, 28, 28])


In [32]:
#첫 번째 합성곱층 선언하기
conv1 = nn.Conv2d(1, 32, 3, padding=1) #Conv2d내의 인지값 (input_channel, output_channel, 커넬 사이즈, 패딩)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [33]:
#두 번째 합성곱층 선언하기
conv2 = nn.Conv2d(32, 64, 3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [34]:
#세 번째 합성곱층 선언하기
pool = nn.MaxPool2d(2) #간단한 선언하면 선언 된 하나의 정수값을 kernel size, stride가 결정되게 한다.
print(pool)

#pool=nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,ceil_mode=False)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [35]:
#선언 된 레이어를 연결하는 과정
#첫번째 레이어 지나가기
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [36]:
#컨벌루셔널 레이어를 거쳐 갈 경우 마지막에 pooling layer가 존재함.
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [37]:
#두 번째 레이어 지나가기
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [38]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [39]:
#view()함수를 사용해서 텐서를 펼치는 작업(텐서를 통합한다.)
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [40]:
fc = nn.Linear(3136, 10) #fully connected : 전체를 연결(기초적인 신경망) 통합된 텐서를 통해 0~9까지의 결과를 갖게 되기 때문에 10개의 output
out = fc(out)
print(out.shape)

#전체 모델은 설정 과정

torch.Size([1, 10])


In [42]:
#위에서 선언한 신경망을 통해서 MNIST(숫자, 데이터셋) 분류 문제 해결하기
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [45]:
device = 'cuda' if torch.cuda.is_available() else'cpu'
#torch.cuda.is_available()함수는 False/True의 boolean형태로 나타나게 되는데, CUDA를 사용 가능할 경우 True를 반환하기 때문에
#if조건을 만족하는 device  = 'cuda'가 되고, 사용이 불가능할 경우 False를 반환하기 때문에 else 조건을 만족하는 device = 'cpu'가 됨.


#GPU 사용시에 고정시킬 시드를 선언
torch.manual_seed(777)

#만약 CPU가 아닌 GPU를 사용해서 훈련을 진행할 경우 고정 된 시드를 통해 GPU로 학습
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [46]:
print(torch.cuda.is_available())

True


In [72]:
#학습에 사용되는 변수(파라미터들: 학습률, 에폭, 배치사이즈 등) 선언
learning_rate = 0.001
training_epoch = 15
batch_size = 100

In [77]:
#MNIST train, test 데이터셋을 torchvision.datasets를 통해 비교적으로 쉽게 가져오기
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [78]:
#데이터로더를 이용하여 배치크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last = True)

In [99]:
class CNN(torch.nn.Module): #CNN구조를 사용한 모델을 선언하기 위한 클래스
    def __init__(self): #초기화 함수, 동시에 모델의 구조를 선언하는 함수
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True )
        
        torch.nn.init.xavier_uniform(self.fc.weight)
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        
        return out

In [100]:
model = CNN().to(device)
#MODEL이라는 객체를 통해서 CNN 클래스가 선언이 되고, 이는. to(device)를 통해 gpu에서 연산을 하게 됨

C:\Users\18\AppData\Local\Temp/ipykernel_4840/3068834243.py:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc.weight)


In [103]:
#비용함수(cost function, loss function)은 크로스엔트로피로 선언
#옵티마이저는 Adam으로 설정
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [109]:
total_batch = len(data_loader)
print('총 배치의 수 :{}'.format(total_batch))

#MNIST dataset은 60,000개의 데이터를 가지고 있는데, batch_size를 100번으로 설정했으므로 총 600의 batch_size를 가지게 됨.

총 배치의 수 :600


In [116]:
#epoch의 수에 따라 반복문을 반복하며 training 실행

for epoch in range(training_epoch):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch +1, avg_cost))

[Epoch:    1] cost = 0.00146525982
[Epoch:    2] cost = 0.0050283419
[Epoch:    3] cost = 0.00094223558
[Epoch:    4] cost = 0.00305797323
[Epoch:    5] cost = 0.00212557614
[Epoch:    6] cost = 0.00117845729
[Epoch:    7] cost = 0.00122580351
[Epoch:    8] cost = 0.00386360474
[Epoch:    9] cost = 0.000777387526
[Epoch:   10] cost = 0.000116185576
[Epoch:   11] cost = 3.47232599e-05
[Epoch:   12] cost = 1.97092122e-05
[Epoch:   13] cost = 1.49744319e-05
[Epoch:   14] cost = 1.20063214e-05
[Epoch:   15] cost = 9.6483891e-06


In [117]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9893999695777893


C:\Users\18\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\18\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
